In [1]:
import ccxt
import numpy as np
#get all ids from ccxt.exchange
ids = ccxt.exchanges

################################################

#remove ids with _ at the start
ids=[id for id in ids if not '_' in id]
exchanges = {}
for id in ids:  # load all markets from all exchange exchanges
    exchange = getattr(ccxt, id)() # instantiate the exchange by id
    exchanges[id] = exchange # save it in a dictionary under its id for future use
    
#######################################################
    
tickerS_ids=[] #initialise the list that contains all ids that are compatible with the 'fetch_tickers' command
nontickerS_ids=[] #initialise the list that contains all ids that are NOT compatible with the 'fetch_tickers' command
all_tickerS={} #initialise the dict that contains all tickers fetched by the 'fetch_tickers' command
for id in ids:
    try:
        tickers=(eval('ccxt.%s().fetch_tickers()' %id))
        tickerS_ids.append(id) #add the id that worked to 'tickerS_ids
        tickers={id:tickers} #put a key of the name "id" around the dict called tickers
        all_tickerS.update(tickers) #append the newly created dictionry to all_tickerS
    except: 
        nontickerS_ids.append(id)
        #print(id,'not compatible with tickers')

outF = open("tickerS_ids.txt", "w")
for line in tickerS_ids:
    outF.write(line)
    outF.write("\n")
outF.close()
outF = open("nontickerS_ids.txt", "w")
for line in nontickerS_ids:
    outF.write(line)
    outF.write("\n")
outF.close()

In [9]:
#check that there is no file called all_tickerS.npy

In [4]:
# Save
np.save('all_tickerS', all_tickerS) 
# Load
read_all_tickerS = np.load('all_tickerS.npy').item()

In [6]:
#make a list of all symbols, exchanges, asks and bid prices. Manipulating a dictionary was too hard.
bysymb=[]
for ex in list(read_all_tickerS.keys()): #first level of keys is exchanges
    for s in list(read_all_tickerS[ex].keys()): #next level of keys is symbols
        ask=read_all_tickerS[ex][s]['ask']
        bid=read_all_tickerS[ex][s]['bid']
        bysymb.append([s,ex,ask,bid]) #####CORRECT THIS. symbols (s) are repeated 
        
#######################################

#process the list to find all symbols, unique symbols, and arbitrable symbols
bysymb_array=np.array(bysymb) # make the list an array
allSymbols=list(bysymb_array[:,0]) #get the first column [:0] of the array which is all the symbols
uniqueSymbols=set(allSymbols) #get rid of duplicate symbols
arbitrableSymbols = sorted([symbol for symbol in uniqueSymbols if allSymbols.count(symbol) > 1]) #make arbitrableSymbols = symbol
#where symbol is allsymbol.count(symbol)>1 in uniquesymbols

############################################

# find the row numbers (indeces) corresponding to each arbitrablesymbol for further processing
indeces={} #initialise indeces as a dictionary
for s in arbitrableSymbols: #for each element of arbitrablesymbols...
    index=[] #initialise index as a list
    index=[i for i, x in enumerate(allSymbols) if x == s] #index = i if x in allsymbols is equal to s (the symbol in arbitrableSymbols)
    indeces.update({s:index}) #save the indeces of each symbol (s) in a dictonary.
    

#########################################



In [7]:
#make a list "check" of all [s,ex,ask,bid] if not NONE and more than 0
tally=[]
arb_ex=[]
for s in list(indeces.keys()): #for each symbol s in indeces...
    ask=[]
    bid=[]
    asks=[]
    bids=[]
    index=[]
    check=[]
    for i in indeces[s]: #for each index in the list of indeces
        if bysymb_array[i,2] != None and bysymb_array[i,3] != None and bysymb_array[i,2] >0 and bysymb_array[i,3]>0:
            ask.append(bysymb_array[i,2]) #get the ith row and find the 2nd and 3rd elements (ask and bid prices)
            bid.append(bysymb_array[i,3])
            check.append(bysymb_array[i,:]) #keep this for checking the raw prices in bysymb_array
        else: ''
####### make a row matrix of asks and column matrix of bids for symbol and ex. Multiply bids with 1/asks
            #index.append(i)                #keep a record of the indeces REPLACE WITH indeces[s
        asks=np.matrix(ask)               #convert asks and bids to matrices and transpose bids 
        bids=np.transpose(np.matrix(bid)) 
        mat=bids.dot(1/asks) #cross product of bids with 1/asks.
        #strange that .dot is a cross product
        #Asks costs me and bid makes me money. So if asks < bids, good
        b = np.matrix(np.where(mat>1.1)) ### THIS FINDS THE ARBITRAGES
        n=0
        for n in range(0, np.shape(b)[1]):
            ask_ex=check[b[1,n]]
            bid_ex=check[b[0,n]]
            arb_ex.append(ask_ex[1])
            arb_ex.append(bid_ex[1])
            tally.append([s,'buy from', ask_ex[1], ask_ex[2], 'sell to', bid_ex[1],bid_ex[3],bid_ex[3]/ask_ex[2]])

In [8]:

tally

[['ADL/BTC',
  'buy from',
  'livecoin',
  1.121e-05,
  'sell to',
  'ccex',
  1.433e-05,
  1.2783229259589652],
 ['AIO/BTC',
  'buy from',
  'ccex',
  1.599e-05,
  'sell to',
  'bitfinex',
  0.000392,
  24.515322076297682],
 ['AIO/BTC',
  'buy from',
  'ccex',
  1.599e-05,
  'sell to',
  'bitfinex2',
  0.00039207,
  24.519699812382736],
 ['AIO/BTC',
  'buy from',
  'ccex',
  1.599e-05,
  'sell to',
  'bitfinex',
  0.000392,
  24.515322076297682],
 ['AIO/BTC',
  'buy from',
  'ccex',
  1.599e-05,
  'sell to',
  'bitfinex2',
  0.00039207,
  24.519699812382736],
 ['AIO/BTC',
  'buy from',
  'ccex',
  1.599e-05,
  'sell to',
  'ethfinex',
  0.00039217,
  24.52595372107567],
 ['ATX/BTC',
  'buy from',
  'livecoin',
  2.7e-07,
  'sell to',
  'coinexchange',
  4.2e-07,
  1.5555555555555556],
 ['BCD/BTC',
  'buy from',
  'bitz',
  0.0016297,
  'sell to',
  'binance',
  0.003357,
  2.059888323004234],
 ['BCD/BTC',
  'buy from',
  'bitz',
  0.0016297,
  'sell to',
  'binance',
  0.003357,
  2.0